In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
X = newdf.to_numpy()
y = df['CO2E'].to_numpy()

In [ ]:
searchSpace = {
    "n_estimators": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    "max_depth": [3, 6, 9],
    "gamma": [0, 0.1, 0.2],
    "learning_rate": [0.05, 0.1, 0.3],
    "min_child_weight": [1, 3, 5]
}
xgb_model = xgb.XGBRFRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.75, random_state=42)

In [ ]:
GS = GridSearchCV(estimator=xgb_model,
                  param_grid= searchSpace,
                  scoring = ["r2", "neg_root_mean_squared_error"],
                  refit ="r2",
                  cv=5,
                  verbose=4)

In [ ]:
d = GS.fit(X_train, y_train)
d